CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
!pip install alibi-detect
!pip install pytorch_tabular[extra]

In [2]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from alibi_detect.cd import TabularDrift

# Added to retrain from B1
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

np.set_printoptions(formatter={"float": "{:12.8f}".format})

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [3]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here

# print(f"df length before dropping 2022-23: {len(df)}")
# df = df[df.year != 2022]
# df = df[df.year != 2023]
# print(f"df length after dropping 2022-23: {len(df)}")

df = df.drop(['full_address', 'nearest_stn'], axis=1)
# print(f"df:\n{df}")

train = df[df.year < 2020]
val = df[df.year == 2020]
test_2022 = df[df.year == 2022]
test_2023 = df[df.year == 2023]

# print(f"test:\n{test}")

continuous_cols = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
categorical_cols = [ "month", "town", "flat_model_type", "storey_range"]

In [4]:
# TODO: Enter your code here
data_config_2022 = DataConfig(
    target=['resale_price'],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

trainer_config_2022 = TrainerConfig(
    auto_lr_find=True,
    batch_size=1024,
    max_epochs=50,
)

model_config_2022 = CategoryEmbeddingModelConfig(
    task="regression",
    layers=50,
)

optimizer_2022 = OptimizerConfig()

tab_model_2022 = TabularModel(
    data_config=data_config_2022,
    model_config=model_config_2022,
    optimizer_config=optimizer_2022,
    trainer_config=trainer_config_2022,
)

tab_model_2022.fit(train=train, validation=val)
result_2022 = tab_model_2022.evaluate(test_2022)
pred_df_2022 = tab_model_2022.predict(test_2022)

# tab_model_2022.save_model("./b1")
# loaded_model = TabularModel.load_from_checkpoint("./b1")

2023-10-06 06:27:03,360 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Global seed set to 42
2023-10-06 06:27:03,417 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-10-06 06:27:03,428 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
2023-10-06 06:27:03,799 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
2023-10-06 06:27:03,873 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
/usr/local/lib/python3.10/dist-packages/pytorch_lightning

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_2d87b4aa-7008-45a9-b1d6-1b8bd923b9f5.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/.lr_find_2d87b4aa-7008-45a9-b1d6-1b8bd923b9f5.ckpt
2023-10-06 

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 K                                                                                                
Total estimated model params size (MB): 0

...

Output()

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       16267038720.0       │
│  test_mean_squared_error  │       16267038720.0       │
└───────────────────────────┴───────────────────────────┘

Output()

In [5]:
# print(f"results:\n{result}")
from sklearn.metrics import r2_score

print(f"{result_2022[0]['test_mean_squared_error']}")
mse = np.sqrt(float(result_2022[0]['test_mean_squared_error']))

pred_2022 = pred_df_2022["resale_price_prediction"].values.tolist()
y_2022 = pred_df_2022['resale_price'].values.tolist()

rsquared = r2_score(y_2022, pred_2022)
print(f"test loss: {result_2022[0]['test_loss']}")
print(f"RMSE: {mse}")
print(f"R Squared: {rsquared}")

16267038720.0
test loss: 16267038720.0
RMSE: 127542.30168849863
R Squared: 0.43884732217645395


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [6]:
# TODO: Enter your code here
data_config_2023 = DataConfig(
    target=['resale_price'],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

trainer_config_2023 = TrainerConfig(
    auto_lr_find=True,
    batch_size=1024,
    max_epochs=50,
)

model_config_2023 = CategoryEmbeddingModelConfig(
    task="regression",
    layers=50,
)

optimizer_2023 = OptimizerConfig()

tab_model_2023 = TabularModel(
    data_config=data_config_2023,
    model_config=model_config_2023,
    optimizer_config=optimizer_2023,
    trainer_config=trainer_config_2023,
)

tab_model_2023.fit(train=train, validation=val)
result_2023 = tab_model_2023.evaluate(test_2023)
pred_df_2023 = tab_model_2023.predict(test_2023)

2023-10-06 06:28:18,330 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Global seed set to 42
2023-10-06 06:28:18,376 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-10-06 06:28:18,384 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
2023-10-06 06:28:18,725 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
2023-10-06 06:28:18,795 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
/usr/local/lib/python3.10/dist-packages/pytorch_lightning

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_054f5985-7f15-46e9-86be-acc7602932f5.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/.lr_find_054f5985-7f15-46e9-86be-acc7602932f5.ckpt
2023-10-06 

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.5 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 K                                                                                                
Total estimated model params size (MB): 0

...

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       24701362176.0       │
│  test_mean_squared_error  │       24701362176.0       │
└───────────────────────────┴───────────────────────────┘

Output()

In [7]:
print(f"{result_2023[0]['test_mean_squared_error']}")
mse = np.sqrt(float(result_2023[0]['test_mean_squared_error']))

pred_2023 = pred_df_2023["resale_price_prediction"].values.tolist()
y_2023 = pred_df_2023['resale_price'].values.tolist()

rsquared = r2_score(y_2023, pred_2023)
print(f"test loss: {result_2023[0]['test_loss']}")
print(f"RMSE: {mse}")
print(f"R Squared: {rsquared}")

24701362176.0
test loss: 24701362176.0
RMSE: 157166.67005443617
R Squared: 0.16212595367206906


> Did model degradation occur for the deep learning model?

\# TODO: \<Enter your answer here\>

Yes model degradation occurred, considering that the R^2 dropped significantly.



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [8]:
# TODO: Enter your code here
train = train.drop(['year'], axis=1)
test_2023 = test_2023.drop(['year'], axis=1)
sample_train = train.sample(n=1000, random_state=1)
sample_test_2023 = test_2023.sample(n=1000, random_state=1)
labels = ['No!', 'Yes!']
categories_per_feature = {}

for cat in categorical_cols:
    categories_per_feature[sample_train.columns.get_loc(cat)] = None

feature_names = sample_train.columns
sample_train_np = sample_train.to_numpy()
sample_test_2023_np = sample_test_2023.to_numpy()

print(f"cat_per_feature: {categories_per_feature}")



cat_per_feature: {0: None, 1: None, 6: None, 9: None}


In [9]:
cd = TabularDrift(sample_train_np, p_val=.05, categories_per_feature=categories_per_feature)
pred = cd.predict(sample_test_2023_np, drift_type='feature')

In [10]:
print(f"data:")
for key, val in pred['data'].items():
    print(f"{key} :{val}")

for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = pred['data']['is_drift'][f]
    stat_val, p_val = pred['data']['distance'][f], pred['data']['p_val'][f]
    print(f'{fname:25s} -- Drift? {labels[is_drift]:5s} -- {stat:5s}: {stat_val:8.3f} -- p-value: {p_val:6.3f}')

data:
is_drift :[1 1 1 1 0 1 1 1 0 0 1]
distance :[468.18978882  43.90776825   0.09900000   0.08900000   0.03100000
   0.07000000  66.85957336   0.14399999   0.04700000  16.58175850
   0.40300000]
p_val :[  0.00000000   0.01110574   0.00010194   0.00067704   0.71045446
   0.01416832   0.00093820   0.00000000   0.21256767   0.27915153
   0.00000000]
threshold :0.05
month                     -- Drift? Yes!  -- Chi2 :  468.190 -- p-value:  0.000
town                      -- Drift? Yes!  -- Chi2 :   43.908 -- p-value:  0.011
dist_to_nearest_stn       -- Drift? Yes!  -- K-S  :    0.099 -- p-value:  0.000
dist_to_dhoby             -- Drift? Yes!  -- K-S  :    0.089 -- p-value:  0.001
degree_centrality         -- Drift? No!   -- K-S  :    0.031 -- p-value:  0.710
eigenvector_centrality    -- Drift? Yes!  -- K-S  :    0.070 -- p-value:  0.014
flat_model_type           -- Drift? Yes!  -- Chi2 :   66.860 -- p-value:  0.001
remaining_lease_years     -- Drift? Yes!  -- K-S  :    0.144 -- p-value: 

> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


\# TODO: \<Enter your answer here\>

Concept Drift led to the model degradation.
Our input features remained the same and yet we cannot predict the resale prices given our input features.

> From your analysis via TabularDrift, which features contribute to this shift?



\# TODO: \<Enter your answer here\>

month, town, dist_to_nearest_stn, dist_to_dhoby, eigenvector_centrality, flat_model_type, remaining_lease_years

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


\# TODO: \<Enter your answer here\>

One way to address model degradation would be to add new data to allow the model to learn the changes in the data. By combining the old data with more recent data, the model might not degrade as much.

From my experiment below, after increasing my training dataset from 2019 and before to 2021 and before, and changing the validation dataset from 2020 to 2022, we can see that the R^2 score increased rather significantly, from originally R^2 = 0.1___ to R^2 = 0.3___

In [11]:
# TODO: Enter your code here
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here

# print(f"df length before dropping 2022-23: {len(df)}")
# df = df[df.year != 2022]
# df = df[df.year != 2023]
# print(f"df length after dropping 2022-23: {len(df)}")

df = df.drop(['full_address', 'nearest_stn'], axis=1)
# print(f"df:\n{df}")

train_new = df[df.year < 2022]
# val = df[df.year == 2020]
val_new = df[df.year == 2022]
test_2023_new = df[df.year == 2023]

# print(f"test:\n{test}")

continuous_cols = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
categorical_cols = [ "month", "town", "flat_model_type", "storey_range"]


In [12]:
# TODO: Enter your code here
data_config_2023_new = DataConfig(
    target=['resale_price'],
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
)

trainer_config_2023_new = TrainerConfig(
    auto_lr_find=True,
    batch_size=1024,
    max_epochs=50,
)

model_config_2023_new = CategoryEmbeddingModelConfig(
    task="regression",
    layers=50,
)

optimizer_2023_new = OptimizerConfig()

tab_model_2023_new = TabularModel(
    data_config=data_config_2023_new,
    model_config=model_config_2023_new,
    optimizer_config=optimizer_2023_new,
    trainer_config=trainer_config_2023_new,
)

tab_model_2023_new.fit(train=train_new, validation=val_new)
result_2023_new = tab_model_2023_new.evaluate(test_2023_new)
pred_df_2023_new = tab_model_2023_new.predict(test_2023_new)

2023-10-06 06:29:12,387 - {pytorch_tabular.tabular_model:105} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Global seed set to 42
2023-10-06 06:29:12,431 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-10-06 06:29:12,440 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
2023-10-06 06:29:12,945 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
2023-10-06 06:29:13,009 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
/usr/local/lib/python3.10/dist-packages/pytorch_lightning

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=100` reached.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.5754399373371567
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_aa1c543f-1c49-4b2e-a999-77e294c3d090.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/.lr_find_aa1c543f-1c49-4b2e-a999-77e294c3d090.ckpt
2023-10-06 

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 K                                                                                                
Total estimated model params size (MB): 0

...

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_loss', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/module.py:493: UserWarning: You called 
`self.log('test_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing 
`Trainer(logger=ALogger(...))`
  rank_zero_warn(

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │       17918609408.0       │
│  test_mean_squared_error  │       17918609408.0       │
└───────────────────────────┴───────────────────────────┘

Output()

In [13]:
# print(f"{result_2023_new[0]['test_mean_squared_error']}")
mse_new = np.sqrt(float(result_2023_new[0]['test_mean_squared_error']))

pred_2023_new = pred_df_2023_new["resale_price_prediction"].values.tolist()
y_2023_new = pred_df_2023_new['resale_price'].values.tolist()

rsquared_new = r2_score(y_2023_new, pred_2023_new)
print(f"New R^2 Results:")
print(f"\ttest loss: {result_2023_new[0]['test_loss']}")
# print(f"\tRMSE_new: {mse_new}")
print(f"\tR Squared: {rsquared_new}")

New R^2 Results:
	test loss: 17918609408.0
	R Squared: 0.39219800278950967


In [14]:
print(f"Old R^2 Results:")
print(f"\ttest loss: {result_2023[0]['test_loss']}")
# print(f"\tRMSE: {mse}")
print(f"\tR Squared: {rsquared}")

Old R^2 Results:
	test loss: 24701362176.0
	R Squared: 0.16212595367206906


### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |